In [1]:
import requests
import env
from requests.models import Request, Response
from env import Client_ID
import pandas as pd
from igdb.igdbapi_pb2 import GameResult
from typing import Dict, List, Optional, Union, cast
import acquire
import json

### Connect to API and scrape the data needed
#### get the access token and 200 response 

In [ ]:
def get_api():   
    url = env.get_db_url()
    response = requests.post(url)
    data = response.json()
    access_token = data['access_token']
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *;'
    r = requests.post('https://api.igdb.com/v4/games', data=data, headers=headers)
    return access_token, r

In [2]:
access_token = acquire.connect_api()

In [ ]:
access_token

In [3]:
tf = acquire.connect_to_games()

In [ ]:
tf

In [4]:
wrapper = acquire.run_wrapper()

In [ ]:
wrapper

In [5]:
df = acquire.get_game_library(wrapper)
df.head()

,id,category,created_at,external_games,first_release_date,genres,name,platforms,release_dates,similar_games,...,aggregated_rating_count,multiplayer_modes,dlcs,ports,expansions,expanded_games,remakes,remasters,forks,standalone_expansions
0,40104,0,1498435200,"[20127, 1988884]",5.364576e+08,[5],Dogou Souken,[52],[91007],"[12364, 27270, 43367, 103281, 103292, 103298, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,85031,0,1517392813,"[216805, 1969751, 2005770]",1.404346e+09,[9],City Mysteries,[37],[138333],"[10603, 19222, 25905, 41349, 86974, 87507, 885...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,99234,0,1524843905,"[603512, 1857496, 1917109]",1.527120e+09,"[31, 32]",Sword of the Black Stone,[6],[188385],"[25646, 29783, 68271, 80916, 96217, 105269, 10...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,148930,0,1621959514,"[2052375, 2057300]",NaN,"[13, 15, 31, 32]",Bok-Bok: A Chicken Dating Sim,NaN,NaN,"[25311, 26145, 28277, 33603, 51577, 65827, 791...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,139538,0,1602368467,[1966278],1.525910e+09,[32],sun machine,[82],[213594],"[11666, 13200, 13660, 17130, 25311, 28070, 348...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.platforms.tail(10).values[0]

## scraping endpoints
### scrape the end points of each category 

In [ ]:
access_token


In [ ]:
df.head()

### platform_families

In [21]:
def get_platform_families():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/platform_families' ,data=data ,  headers=headers)
    platform_families = wrapper.api_request('platform_families', 'fields *;limit 500;')
    x = json.loads(platform_families)
    platform_families =pd.DataFrame(x)
    return platform_families

In [22]:
platform_families = get_platform_families()
platform_families.head()

,id,name,slug,checksum
0,5,Nintendo,nintendo,398e5c87-f7db-6265-6da6-433441385208
1,4,Linux,linux,be8b89fa-823c-790a-1d5a-7e74315c1b27
2,2,Xbox,xbox,5bb9bcec-4aa7-45b8-129b-a6f88932942e
3,3,Sega,sega,bd1a9cc4-3632-33b0-ff4b-feab8cd15001
4,1,PlayStation,playstation,eadc0208-7972-9364-88b9-d2a9de75a998


In [ ]:
df.columns

### platform_version_release_dates

In [19]:
def get_platform_version_release():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/platform_version_release_dates' ,data=data ,  headers=headers)
    platform_version_release_dates = wrapper.api_request('platform_version_release_dates', 'fields *;limit 500;')
    x = json.loads(platform_version_release_dates)
    platform_version_release_dates =pd.DataFrame(x)
    return platform_version_release_dates

In [20]:
platform_version_release_dates = get_platform_version_release()
platform_version_release_dates

,id,category,date,human,m,region,y,checksum,updated_at
0,555,2,567907200,1987,12,8,1987,186c07ec-0d93-dc0f-6c4a-5fdb5a6138fa,NaN
1,338,0,1189555200,"Sep 12, 2007",9,3,2007,15e5b0aa-5bb6-ef30-ee85-0ef94cb3581f,NaN
2,334,0,1125532800,"Sep 01, 2005",9,7,2005,a4c921d5-1894-9a40-d20f-05959cd69a38,NaN
3,331,0,1415923200,"Nov 14, 2014",11,3,2014,3ebb8dc8-b8fa-f11a-b729-228b9dac35c4,NaN
4,333,0,1389830400,"Jan 16, 2014",1,7,2014,a15ecea5-cb71-916f-5bec-66ad7f571b19,NaN
...,...,...,...,...,...,...,...,...,...
393,661,2,725760000,1992,12,8,1992,d6cb796b-0679-d99b-30d8-decdf573187d,1.660320e+09
394,662,0,945043200,"Dec 13, 1999",12,5,1999,6fa54af0-85f8-088f-294c-d95dbbbb2a70,1.660322e+09
395,663,2,851990400,1996,12,2,1996,86254855-ee73-231a-6fab-9317893d2b30,1.660323e+09
396,671,0,1456272000,"Feb 24, 2016",2,8,2016,3a26664a-ee1c-7d8d-58ad-24fcac7cb66f,1.663687e+09


In [ ]:
# drop columns not needed
platform_version_release_dates = platform_version_release_dates.drop(columns=['category', 'date', 'm', 'y', 'region', 'updated_at', 'checksum', 'updated_at'])
platform_version_release_dates.head()


In [ ]:
df.head()

In [ ]:
version_dates_dict[9] = 'not_available'
version_dates_dict[384] = 'not_available'
version_dates_dict[386] = 'not_available'


In [ ]:
version_dates_list = platform_version_release_dates[['game' , 'human']].sort_values(by='game').reset_index(drop=True)
version_dates_dict = platform_version_release_dates.set_index('game').to_dict()['human']

In [ ]:
def test_funct3(random_list):
    if type(random_list) == list:
        return [version_dates_dict[i] for i in random_list]
    else:  
        return "Not available"

In [ ]:
platforms.set_index('Locality', inplace=True)
data.set_index('Locality', inplace=True)


In [ ]:
df['release_dates'] = df.release_dates.apply(test_funct3)
df.head()

In [ ]:
all_release_dates = pd.merge(release_dates, platform_version_release_dates, how = 'inner', on="human")

In [ ]:
#pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
df.columns

### platform_version_companies

In [15]:
def get_platform_version_companies():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/platform_version_companies' ,data=data ,  headers=headers)
    platform_version_companies = wrapper.api_request('platform_version_companies', 'fields *;limit 500;')
    x = json.loads(platform_version_companies)
    platform_version_companies =pd.DataFrame(x)
    return platform_version_companies

In [16]:
platform_version_companies=get_platform_version_companies()
platform_version_companies.head()

,id,company,developer,manufacturer,checksum,comment
0,286,11986,True,True,e5035f31-713a-1036-2968-dd353b74db3b,NaN
1,283,10922,False,True,6cd3e57c-2757-cc07-94a6-35ff7085e4c9,NaN
2,282,14429,False,True,172b6f24-a49c-c304-6fb3-78f6bf120a19,NaN
3,281,12385,True,False,d6051460-e7dc-4e77-80ca-e5df378cd304,NaN
4,280,19831,True,True,2f509e1b-75d2-36db-4129-4bf89d50343b,NaN


### platforms

In [6]:
def get_platforms():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/platforms' ,data=data ,  headers=headers)
    platforms = wrapper.api_request('platforms', 'fields *;limit 500;')
    x = json.loads(platforms)
    platforms =pd.DataFrame(x)
    return platforms

In [7]:
platform = get_platforms()
platform.shape

(197, 16)

In [8]:
platform.head()

,id,alternative_name,category,created_at,name,platform_logo,slug,updated_at,url,versions,websites,checksum,generation,platform_family,abbreviation,summary
0,158,Commodore Dynamic Total Vision,6.0,1510012800,Commodore CDTV,292.0,commodore-cdtv,1522972800,https://www.igdb.com/platforms/commodore-cdtv,[223],[26],5d48648e-283c-e2df-df7d-6d3cbc7e0a58,NaN,NaN,NaN,NaN
1,339,Kids Computer Pico,1.0,1595808000,Sega Pico,NaN,sega-pico,1595808000,https://www.igdb.com/platforms/sega-pico,[456],NaN,6e867a5b-a071-fc7b-945b-a0622f64a4bc,4.0,3.0,NaN,NaN
2,8,PS2,1.0,1297555200,PlayStation 2,254.0,ps2,1414972800,https://www.igdb.com/platforms/ps2,"[58, 114]",NaN,be3e025c-d4ed-b795-443c-a8cb55fbfe17,6.0,1.0,PS2,NaN
3,39,NaN,4.0,1317686400,iOS,248.0,ios,1391644800,https://www.igdb.com/platforms/ios,[43],NaN,fefe4a2b-7c90-0e89-e811-902ea1cf2b58,NaN,NaN,iOS,NaN
4,94,NaN,6.0,1414195200,Commodore Plus/4,310.0,c-plus-4,1468454400,https://www.igdb.com/platforms/c-plus-4,[108],NaN,ea138789-6bbe-bcca-dff5-7e20e762f65a,NaN,NaN,C+4,NaN


In [ ]:
# see if there is a number in id that matches the big df
platform.loc[platform['id'] == 5]

In [ ]:
platform = platform.drop(columns=['alternative_name', 'category', 'created_at', 'platform_logo', 'updated_at', 'url', 'checksum', 'versions', 'websites', 'generation', 'platform_family', 'abbreviation', 'summary'])
platform.head()


In [ ]:
# looking at the dictionary
print(df.get(92, "Not found"))

In [ ]:
df.platforms.iloc[2]

In [9]:
platformlist = platform[['id' , 'name']].sort_values(by='id').reset_index(drop=True)


In [10]:
platform_dict = platformlist.set_index('id').to_dict()['name']

In [13]:
platform_dict[92] = 'not_available'

In [ ]:
platform_dict['0'] = 'not_available'

In [ ]:
import numpy as np
platform.head()

In [ ]:
platform_dict

In [11]:
def test_funct(random_list):
    if type(random_list) == list:
        return [platform_dict[i] for i in random_list]
    else:  
        return "Not available"

In [ ]:
print(platform.get('id', "Not found"))

In [ ]:
df.platforms.info(10)

In [14]:
df['platforms'] = df.platforms.apply(test_funct)
df.head()

,id,category,created_at,external_games,first_release_date,genres,name,platforms,release_dates,similar_games,...,aggregated_rating_count,multiplayer_modes,dlcs,ports,expansions,expanded_games,remakes,remasters,forks,standalone_expansions
0,40104,0,1498435200,"[20127, 1988884]",5.364576e+08,[5],Dogou Souken,[Arcade],[91007],"[12364, 27270, 43367, 103281, 103292, 103298, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,85031,0,1517392813,"[216805, 1969751, 2005770]",1.404346e+09,[9],City Mysteries,[Nintendo 3DS],[138333],"[10603, 19222, 25905, 41349, 86974, 87507, 885...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,99234,0,1524843905,"[603512, 1857496, 1917109]",1.527120e+09,"[31, 32]",Sword of the Black Stone,[PC (Microsoft Windows)],[188385],"[25646, 29783, 68271, 80916, 96217, 105269, 10...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,148930,0,1621959514,"[2052375, 2057300]",NaN,"[13, 15, 31, 32]",Bok-Bok: A Chicken Dating Sim,Not available,NaN,"[25311, 26145, 28277, 33603, 51577, 65827, 791...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,139538,0,1602368467,[1966278],1.525910e+09,[32],sun machine,[Web browser],[213594],"[11666, 13200, 13660, 17130, 25311, 28070, 348...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.loc[df['platforms'] == '3ds']

In [ ]:
df.loc[df['platforms'].notnull()]

### release_dates

In [17]:
def get_release_dates():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/release_dates' ,data=data ,  headers=headers)
    release_dates = wrapper.api_request('release_dates', 'fields *;limit 500;')
    x = json.loads(release_dates)
    release_dates =pd.DataFrame(x)
    return release_dates

In [18]:
release_dates = get_release_dates()
release_dates.shape

(500, 12)

In [ ]:
release_dates = release_dates.drop(columns=['category', 'date', 'm', 'region', 'updated_at', 'y', 'checksum', 'created_at', 'platform'])
release_dates.head()

In [ ]:
release_dates

In [ ]:
# add to dictionary
release_dict[91007] = "not-available"
release_dict[138333] = "not-available"
release_dict[188385] = "not-available"
release_dict[213594] = "not-available"
release_dict[77755] = "not-available"


In [ ]:
releaselist = release_dates[['game' , 'human']].sort_values(by='game').reset_index(drop=True)
release_dict = releaselist.set_index('game').to_dict()['human']

In [ ]:
release_dates.head()

In [ ]:
def test_funct2(random_list):
    if type(random_list) == list:
        return [release_dict[i] for i in random_list]
    else:  
        return "Not available"

In [ ]:
release_dict

In [ ]:
df.release_dates.head()

In [ ]:
df['release_dates'] = pd.DataFrame(df.release_dates.apply(test_funct2))
df

In [ ]:
release_dict

In [ ]:
# add 0 and description to dictionary
release_dict["0"] = "not-available"

### platform_versions

In [ ]:
def get_platform_versions():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/platform_versions' ,data=data ,  headers=headers)
    platform_versions = wrapper.api_request('platform_versions', 'fields *;limit 500;')
    x = json.loads(platform_versions)
    platform_versions=pd.DataFrame(x)
    return platform_versions

In [ ]:
platform_versions = get_platform_versions()
platform_versions.shape

In [ ]:
platform_versions.head()

In [ ]:
df.columns

### player_perspectives

In [ ]:
def get_player_perspectives():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/player_perspectives' ,data=data ,  headers=headers)
    player_perspectives = wrapper.api_request('player_perspectives', 'fields *;limit 500;')
    x = json.loads(player_perspectives)
    player_perspectives=pd.DataFrame(x)
    return player_perspectives

In [ ]:
player_perspectives = get_player_perspectives()
player_perspectives.shape

In [ ]:
player_perspectives.head()

In [ ]:
player_perspectives = player_perspectives.drop(columns=['created_at','updated_at', 'url', 'checksum'])
player_perspectives.head()


In [ ]:
player_perspectives_list = player_perspectives[['id' , 'name']].sort_values(by='id').reset_index(drop=True)
player_perspectives_dict = player_perspectives.set_index('id').to_dict()['name']

In [ ]:
def test_funct4(random_list):
    if type(random_list) == list:
        return [player_perspectives_dict[i] for i in random_list]
    else:  
        return "Not available"

In [ ]:
df['player_perspectives'] = pd.DataFrame(df.player_perspectives.apply(test_funct4))
df

In [ ]:
df.columns

### themes

In [ ]:
def get_themes():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/themes' ,data=data ,  headers=headers)
    themes = wrapper.api_request('themes', 'fields *;limit 500;')
    x = json.loads(themes)
    themes=pd.DataFrame(x)
    return themes

In [ ]:
themes = get_themes()
themes.shape

In [ ]:
themes.head()

In [ ]:
themes_list = themes[['id' , 'name']].sort_values(by='id').reset_index(drop=True)
themes_dict = themes.set_index('id').to_dict()['name']

In [ ]:
def test_funct5(random_list):
    if type(random_list) == list:
        return [themes_dict[i] for i in random_list]
    else:  
        return "Not available"

In [ ]:
df['themes'] = pd.DataFrame(df.themes.apply(test_funct5))
df

### Read the CSV for games 

In [ ]:
df = pd.read_csv('games.csv', index_col='Unnamed: 0')
df.head()

In [ ]:
#pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
df.loc[df['franchises'].notnull()]

In [ ]:
df.platforms.info()

In [ ]:
df = df.drop(columns=['websites', 'created_at', 'updated_at', 'artworks', 'cover','screenshots', 'videos', 'checksum', 'alternative_names', 'rating_count', 'total_rating_count', 'version_parent', 'hypes', 'franchises', 'follows', 'aggregated_rating_count'])
df.shape

In [ ]:
df.columns

In [ ]:
df.rename(columns={'slug': 'lower_case_name', 'aggregated_rating': 'critic_rating'}, inplace=True)
df.head()

In [ ]:
df.rating.isnull().sum()

In [ ]:
df.loc[df['rating'].isnull()]

In [ ]:
df.release_dates.info()

In [ ]:
df.platforms.value_counts()

### Dataframe with ratings

In [ ]:
df2 = df[df['rating'].notnull()]
df2.head()

In [ ]:
df2.rating.isnull().sum()

### Dataframe with nulls in rating column 

In [ ]:
df3 = df[df['rating'].isnull()]
df3.shape